In [1]:
import time
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import undetected_chromedriver as uc
import urllib.parse
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Enter desired job titles
user_input_job_title = input('Enter job titles (separated by commas): ').split(',')
user_input_job_title = [title.strip() for title in user_input_job_title]  # Remove whitespace
print(f"Desired job titles: {user_input_job_title}")

Nhập tên công việc (cách nhau bởi dấu phẩy):  data scientist


Công việc cần tìm: ['data scientist']


In [7]:
# Create search URL for LinkedIn Vietnam
b = []
for i in user_input_job_title:
    x = i.split()
    y = '%20'.join(x)
    b.append(y)
job_title = '%2C%20'.join(b)
print(f"Search keywords: {job_title}")

Từ khóa tìm kiếm: data%20scientist


In [9]:
# URL for LinkedIn Vietnam (geoId=104195383 is the ID of Vietnam)
link = f"https://www.linkedin.com/jobs/search?keywords={job_title}&location=Vietnam&locationId=&geoId=104195383&f_TPR=r604800&position=1&pageNum=0"
print(f"Search link: {link}")

Link tìm kiếm: https://www.linkedin.com/jobs/search?keywords=data%20scientist&location=Vietnam&locationId=&geoId=104195383&f_TPR=r604800&position=1&pageNum=0


In [62]:
# First time running:
# Initialize Chrome with a custom profile (store session here)
options = uc.ChromeOptions()
options.user_data_dir = "selenium_profile"  # directory to store session

driver = uc.Chrome(options=options)

# Open LinkedIn login page
driver.get("https://www.linkedin.com/login")
print("Please log in to LinkedIn manually in the newly opened browser.")

# Wait until you finish logging in
input("After logging in, press Enter in Jupyter to continue...")

# Access the jobs page after login
driver.get(link)

# Next time, you only need to run from here:

# options = uc.ChromeOptions()
# options.user_data_dir = "selenium_profile"  # use the saved session

# driver = uc.Chrome(options=options)
# driver.get(link)

🔐 Hãy đăng nhập LinkedIn thủ công trong trình duyệt vừa mở.


✅ Sau khi đăng nhập xong, nhấn Enter trong Jupyter để tiếp tục... 


In [13]:
def get_jobs_method_2_javascript(driver, base_link, max_pages=1, scroll_times=12, expected_jobs=25):
    all_companies = []
    all_job_titles = []
    all_locations = []
    all_urls = []

    for page_num in range(1, max_pages + 1):
        print(f"\n--- Loading page {page_num} ---")
        current_link = base_link if page_num == 1 else f"{base_link}&start={25 * (page_num - 1)}"

        try:
            driver.get(current_link)
            time.sleep(5)

            # Scroll through the job list to load job cards
            try:
                scroll_container = driver.find_element(By.XPATH, "//div[contains(@class, 'jobs-search-results-list')]")
                for _ in range(scroll_times):
                    driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", scroll_container)
                    time.sleep(1.5)
            except Exception:
                pass

            # Wait for job cards to render
            start_time = time.time()
            while time.time() - start_time < 30:
                job_cards = driver.find_elements(By.XPATH, "//li[@data-occludable-job-id]")
                if len(job_cards) >= expected_jobs:
                    break
                time.sleep(2)

            # Scroll each job card into view to force rendering
            job_cards = driver.find_elements(By.XPATH, "//li[@data-occludable-job-id]")
            for job_card in job_cards:
                try:
                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant', block: 'center'});", job_card)
                    time.sleep(0.5)
                except:
                    pass

            # Extract data using JavaScript
            js_script = """
            var jobCards = document.querySelectorAll('li[data-occludable-job-id]');
            var results = [];

            jobCards.forEach(function(card) {
                try {
                    var titleEl = card.querySelector('a.job-card-container__link span[aria-hidden="true"]') ||
                                  card.querySelector('h3');

                    var companyEl = card.querySelector('div.artdeco-entity-lockup__subtitle span') ||
                                    card.querySelector('.job-card-container__company-name');

                    var locationEl = card.querySelector('ul.job-card-container__metadata-wrapper li span[dir="ltr"]') ||
                                     card.querySelector('span.job-card-container__metadata-item') ||
                                     card.querySelector('.job-card-container__metadata-item');

                    var urlEl = card.querySelector('a.job-card-container__link');

                    var rawUrl = urlEl ? urlEl.getAttribute("href") : '';
                    var fullUrl = rawUrl.startsWith("http") ? rawUrl : "https://www.linkedin.com" + rawUrl;

                    var title = titleEl ? titleEl.textContent.trim() : '';
                    var company = companyEl ? companyEl.textContent.trim() : '';
                    var location = locationEl ? locationEl.textContent.trim() : '';

                    if (title && company && location) {
                        results.push({
                            title: title,
                            company: company,
                            location: location,
                            url: fullUrl
                        });
                    }

                } catch (e) {
                    // Skip errors
                }
            });

            return results;
            """

            job_data = driver.execute_script(js_script)
            print(f"Number of jobs collected: {len(job_data)}")

            for idx, job in enumerate(job_data):
                all_job_titles.append(job['title'])
                all_companies.append(job['company'])
                all_locations.append(job['location'])
                all_urls.append(job['url'])

                print(f"[{idx+1}] {job['title']} | {job['company']} | {job['location']}")

        except Exception as e:
            print(f"Error on page {page_num}: {e}")
            continue

    return all_companies, all_job_titles, all_locations, all_urls

In [15]:
company_name, job_titles, company_location, job_url = get_jobs_method_2_javascript(driver, link, max_pages=3)


--- Đang tải trang 1 ---
Số job thu thập được: 7
[1] Data Scientist Intern | Procter & Gamble | Thuận An, Binh Duong, Vietnam
[2] Data Scientist | Hitachi Digital Services | Củ Chi, Ho Chi Minh City, Vietnam (On-site)
[3] Data Scientist – Strong in Algorithms & Coding | LARION | Ho Chi Minh City, Vietnam (Hybrid)
[4] Binance Accelerator Program - Research Data Analyst | Binance | Ho Chi Minh City, Ho Chi Minh City, Vietnam (Remote)
[5] Analyst, Business Intelligence - HCM | Ninja Van | Ho Chi Minh City, Ho Chi Minh City, Vietnam (On-site)
[6] Data Processing Specialist (Reference Data) | NielsenIQ | Ho Chi Minh City, Ho Chi Minh City, Vietnam
[7] (Fresher to Junior/ Middle Level) Risk Strategist | Zalopay | Ho Chi Minh City, Vietnam (On-site)

--- Đang tải trang 2 ---
Số job thu thập được: 25
[1] Data Processing Specialist (Reference Data) - (10 months contractor) | NielsenIQ | Ho Chi Minh City, Ho Chi Minh City, Vietnam (Hybrid)
[2] Data Engineer at CellphoneS | ĐÚNG NGƯỜI ĐÚNG VIỆC 

In [28]:
driver.quit()

In [18]:
print(f"\nTotal from all pages:")
print(f"Found {len(company_name)} companies")
print(f"Found {len(job_titles)} job titles")
print(f"Found {len(company_location)} locations")
print(f"Found {len(job_url)} job URLs")


Tổng cộng từ tất cả các trang:
Tìm thấy 57 công ty
Tìm thấy 57 tiêu đề công việc
Tìm thấy 57 địa điểm
Tìm thấy 57 URL công việc


In [46]:
# Create DataFrame
min_length = min(len(company_name), len(job_titles), len(company_location), len(job_url))
df = pd.DataFrame({
    'Company Name': company_name[:min_length],
    'Job Title': job_titles[:min_length],
    'Location': company_location[:min_length],
    'Website URL': job_url[:min_length]
})

df

,Company Name,Job Title,Location,Website URL
0,Procter & Gamble,Data Scientist Intern,"Thuận An, Binh Duong, Vietnam",https://www.linkedin.com/jobs/view/4259882267/...
1,Hitachi Digital Services,Data Scientist,"Củ Chi, Ho Chi Minh City, Vietnam (On-site)",https://www.linkedin.com/jobs/view/4260632513/...
2,LARION,Data Scientist – Strong in Algorithms & Coding,"Ho Chi Minh City, Vietnam (Hybrid)",https://www.linkedin.com/jobs/view/4262351375/...
3,Binance,Binance Accelerator Program - Research Data An...,"Ho Chi Minh City, Ho Chi Minh City, Vietnam (R...",https://www.linkedin.com/jobs/view/4260683439/...
4,Ninja Van,"Analyst, Business Intelligence - HCM","Ho Chi Minh City, Ho Chi Minh City, Vietnam (O...",https://www.linkedin.com/jobs/view/4247339188/...
5,NielsenIQ,Data Processing Specialist (Reference Data),"Ho Chi Minh City, Ho Chi Minh City, Vietnam",https://www.linkedin.com/jobs/view/4262357749/...
6,Zalopay,(Fresher to Junior/ Middle Level) Risk Strategist,"Ho Chi Minh City, Vietnam (On-site)",https://www.linkedin.com/jobs/view/4262352885/...
7,NielsenIQ,Data Processing Specialist (Reference Data) - ...,"Ho Chi Minh City, Ho Chi Minh City, Vietnam (H...",https://www.linkedin.com/jobs/view/4263178992/...
8,ĐÚNG NGƯỜI ĐÚNG VIỆC Community,Data Engineer at CellphoneS,"Phường Chí Minh, Hai Duong, Vietnam (On-site)",https://www.linkedin.com/jobs/view/4264253207/...
9,TikTok,eCommerce Strategist Project Intern (TikTok Sh...,"Ho Chi Minh City, Ho Chi Minh City, Vietnam",https://www.linkedin.com/jobs/view/4262156118/...


In [48]:
# Function to filter job titles
def job_title_filter(x, user_input_job_title):
    if pd.isna(x) or x == '':
        return np.nan

    s = [i.lower() for i in user_input_job_title]
    suggestion = []
    for i in s:
        suggestion.extend(i.split())

    s = x.split()
    a = [i.lower() for i in s]

    intersection = list(set(suggestion).intersection(set(a)))
    return x if len(intersection) >= 1 else np.nan  # Condition relaxed from >1 to >=1

In [50]:
# Apply the filter
df['Job Title'] = df['Job Title'].apply(lambda x: job_title_filter(x, user_input_job_title))
df = df.dropna()
df.reset_index(drop=True, inplace=True)

print(f"After filtering: {len(df)} matching jobs")

Sau khi lọc: 24 công việc phù hợp


In [52]:
# Select up to the first 5 jobs (since there may be more pages)
df = df.iloc[:5, :]

In [68]:
def get_job_description(driver, job_link):
    try:
        print(f"[INFO] Visiting: {job_link}")
        driver.get(job_link)

        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        time.sleep(2)

        # Click "See more" if available and visible
        try:
            see_more_btn = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "button[aria-label*='see more description']"))
            )
            if see_more_btn.is_displayed() and see_more_btn.is_enabled():
                print("[INFO] Clicking the 'See more' button...")
                driver.execute_script("arguments[0].click();", see_more_btn)
                time.sleep(1.5)
                print("[SUCCESS] 'See more' clicked")
            else:
                print("[INFO] 'See more' button not visible – possibly not needed")
        except TimeoutException:
            print("[INFO] 'See more' button not found")

        # Extract job description content from possible selectors
        desc_text = ""
        possible_selectors = [
            "div.jobs-box__html-content",
            "div.jobs-description__container",
            "div.jobs-description-content__text"
        ]

        for selector in possible_selectors:
            try:
                desc_elem = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, selector))
                )
                desc_text = desc_elem.text.strip()
                if desc_text:
                    break
            except TimeoutException:
                continue

        if desc_text:
            print("[SUCCESS] Job description retrieved")
            return desc_text
        else:
            print("[WARNING] No content found")
            return "Description content not found"

    except TimeoutException:
        return "Timeout"
    except Exception as e:
        return f"Error: {str(e)}"


def scrape_all_job_descriptions(df):
    """
    Loop through all job URLs and retrieve job descriptions.

    Args:
        df (pd.DataFrame): DataFrame containing a 'Website URL' column

    Returns:
        pd.DataFrame: DataFrame with an added 'Job Description' column
    """
    options = uc.ChromeOptions()
    options.user_data_dir = "selenium_profile"  # use saved session
    options.add_argument("--start-maximized")
    # options.add_argument("--headless")  # Uncomment if you want to run browser in background

    driver = uc.Chrome(options=options)

    website_urls = df['Website URL'].tolist()
    job_descriptions = []

    print("Fetching job descriptions...")
    for i, url in enumerate(website_urls):
        print(f"\n=== Job {i+1}/{len(website_urls)} ===")
        try:
            desc = get_job_description(driver, url)
        except Exception as e:
            desc = f"Error: {str(e)}"
        job_descriptions.append(desc)

    df['Job Description'] = job_descriptions
    return df


In [70]:
df_result = scrape_all_job_descriptions(df)

print(f"\nDone! Found {len(df_result)} jobs in Vietnam:")
print("=" * 80)

Đang lấy mô tả công việc...

=== Công việc 1/5 ===
[INFO] Đang truy cập: https://www.linkedin.com/jobs/view/4259882267/?eBP=CwEAAAGX84lwZyTj5FsSGLBcwF3RQoafGX2k2FuMk8rcC-0-y4Dm4ukGI-ksrNypyU_APuxi61vO64oLHPrP58bbsosVIPYD4fAuTtLyrrCd7y8NeuNFu_8nptuv7Dwyn0gAj6-T5NLuQgfTsHFRkDSE_YaOnBVTJT4c13WfqHpXozACQiy6MU6ycv07r5CHpXW_KrJykYQ9Hg8jt8qkh_QBp5W5DIHzIx2fb0rUGAhJmMqPtJRh0SLk-sZowlISrc8-4ikwEolh5l4YCQhhWFWJ4TyivMj6CkQva3_ayIfXMKcqXmPVcvdzGWM976D_GRGAT6AYneRq30UF78SJ3jt1VR7lDaQdbqdfWjSR_kVlgYeY2X4g9A_7xNiK_WDmNrC1RuGE33HFVakrNGTxSpsptCy4SruhoLQJXxHg-pMcIGvnvZKjLPB8Y5-DgbxxiObr086uqqUL8rdA-oHWUKOUDHTOidBvXXLw4i45fRVAk1mMwoSyWICjCEQpiSaDXVCC_nwG4v5XK91aLGXx&refId=gvlvIEsAPagbdHW11ctkmQ%3D%3D&trackingId=vY9oSZNiZ5nJKDspw7qN3w%3D%3D&trk=flagship3_search_srp_jobs
[INFO] Đang click nút 'See more'...
[SUCCESS] Đã click 'See more'
[SUCCESS] Lấy được mô tả công việc

=== Công việc 2/5 ===
[INFO] Đang truy cập: https://www.linkedin.com/jobs/view/4260632513/?eBP=CwEAAAGX84lwZ52okkUtDWIeKbyUn3PMe8WLxz8WT

In [72]:
df_result

,Company Name,Job Title,Location,Website URL,Job Description
0,Procter & Gamble,Data Scientist Intern,"Thuận An, Binh Duong, Vietnam",https://www.linkedin.com/jobs/view/4259882267/...,About the job\nJob Location\n\nBINH DUONG PLAN...
1,Hitachi Digital Services,Data Scientist,"Củ Chi, Ho Chi Minh City, Vietnam (On-site)",https://www.linkedin.com/jobs/view/4260632513/...,About the job\nOur company\n\nWe’re Hitachi Di...
2,LARION,Data Scientist – Strong in Algorithms & Coding,"Ho Chi Minh City, Vietnam (Hybrid)",https://www.linkedin.com/jobs/view/4262351375/...,"About the job\nLARION, a global software outso..."
3,Binance,Binance Accelerator Program - Research Data An...,"Ho Chi Minh City, Ho Chi Minh City, Vietnam (R...",https://www.linkedin.com/jobs/view/4260683439/...,About the job\nBinance is the global blockchai...
4,NielsenIQ,Data Processing Specialist (Reference Data),"Ho Chi Minh City, Ho Chi Minh City, Vietnam",https://www.linkedin.com/jobs/view/4262357749/...,About the job\nCompany Description\n\nNIQ is t...


In [74]:
# Display results
for i in range(len(df_result)):
    print(f"\n--- JOB {i+1} ---")
    print(f"Company Name  : {df_result.iloc[i, 0]}")
    print(f"Job Title     : {df_result.iloc[i, 1]}")
    print(f"Location      : {df_result.iloc[i, 2]}")
    print(f"URL           : {df_result.iloc[i, 3]}")
    print(f"Description   : {df_result.iloc[i, 4][:500]}..." if len(df.iloc[i, 4]) > 500 else f"Description   : {df.iloc[i, 4]}")
    print("-" * 80)


--- CÔNG VIỆC 1 ---
Tên công ty   : Procter & Gamble
Tiêu đề CV    : Data Scientist Intern
Địa điểm      : Thuận An, Binh Duong, Vietnam
URL           : https://www.linkedin.com/jobs/view/4259882267/?eBP=CwEAAAGX84lwZyTj5FsSGLBcwF3RQoafGX2k2FuMk8rcC-0-y4Dm4ukGI-ksrNypyU_APuxi61vO64oLHPrP58bbsosVIPYD4fAuTtLyrrCd7y8NeuNFu_8nptuv7Dwyn0gAj6-T5NLuQgfTsHFRkDSE_YaOnBVTJT4c13WfqHpXozACQiy6MU6ycv07r5CHpXW_KrJykYQ9Hg8jt8qkh_QBp5W5DIHzIx2fb0rUGAhJmMqPtJRh0SLk-sZowlISrc8-4ikwEolh5l4YCQhhWFWJ4TyivMj6CkQva3_ayIfXMKcqXmPVcvdzGWM976D_GRGAT6AYneRq30UF78SJ3jt1VR7lDaQdbqdfWjSR_kVlgYeY2X4g9A_7xNiK_WDmNrC1RuGE33HFVakrNGTxSpsptCy4SruhoLQJXxHg-pMcIGvnvZKjLPB8Y5-DgbxxiObr086uqqUL8rdA-oHWUKOUDHTOidBvXXLw4i45fRVAk1mMwoSyWICjCEQpiSaDXVCC_nwG4v5XK91aLGXx&refId=gvlvIEsAPagbdHW11ctkmQ%3D%3D&trackingId=vY9oSZNiZ5nJKDspw7qN3w%3D%3D&trk=flagship3_search_srp_jobs
Mô tả         : About the job
Job Location

BINH DUONG PLANT

Job Description

Your Team

You will be based in P&G Binh Duong Plant.

What Success Looks Like